# Label resolver development

In [18]:
import csv
import re

In [19]:
word2label_file = '/home/anjani/distant-PICO/CandidateGeneration/ResultInspection/resolve_annot_corpus.tsv'

In [20]:
total_words = []
with open(word2label_file, 'r') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        #print(row[1:])
        removeSpecialChars = re.sub("[!@#$%^&*()[]{};:,./<>?\|`~-=_+]", " ", row[1])
        total_words.append( removeSpecialChars )

In [21]:
len(set(total_words))

37874